## 1. Cargar tuits

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
sample_rate = 0.03
chunk_size = 10000  # Adjust the chunk size as needed
all_chunks = []

for i in range(1,8):
    chunks = pd.read_csv(f'data/tweets_batch_00{i}.tsv', sep='\t',
                         usecols=["tweet_id", "user_id", "full_text"],
                         engine='python',
                         on_bad_lines='skip',
                         skiprows=lambda i: i>0 and random.random() > sample_rate,
                         chunksize=chunk_size)
    all_chunks += chunks

df = pd.concat(all_chunks, ignore_index=True)

for f in ["user_id"]:
    df[f] = df[f].astype("Int64")

In [3]:
df_full = df

In [4]:
df_full.columns

Index(['tweet_id', 'user_id', 'full_text'], dtype='object')

In [5]:
df_full.shape

(184593, 3)

In [6]:
# df = df_full.sample(10000)
df = df_full

In [7]:
len(df)

184593

In [83]:
df.to_csv("data/covid_twitter_sample_topic_model.csv", index=False)

In [8]:
docs = df.full_text.dropna().values.tolist()

In [9]:
docs[:2]

['App de COVID-19 pudo ser un mero sitio web informativo\n \nhttps://t.co/NkzRDGJNKa',
 'Desafíos y oportunidades por el COVID-19 https://t.co/nEajnYBi7L']

In [10]:
docs[0]

'App de COVID-19 pudo ser un mero sitio web informativo\n \nhttps://t.co/NkzRDGJNKa'

3. Topic modeling

## Hashtags populares

In [11]:
import re

import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return str(only_ascii)

def extract_hashtags(s):
    return [
        remove_accents(ht.lower())
        for ht in re.findall(r"#(\w+)", s)
    ]

In [12]:
df.head()

,tweet_id,user_id,full_text
0,1249125075179016192,1047598652510937088,App de COVID-19 pudo ser un mero sitio web inf...
1,1249125077020532740,233033221,Desafíos y oportunidades por el COVID-19 https...
2,1249125082171146246,3433643799,Ganamos el ultimo UHC del evento Benefico cont...
3,1249125108284882945,1413747024,"En toda circunstancia, en todo lugar, siempre ..."
4,1249125110398795782,15666807,El epidemiólogo Abdel Rodriguez Rivera habla c...


In [13]:
df["full_text"] = df.full_text.astype(str)

In [14]:
df["hashtags"] = df.full_text.apply(extract_hashtags)

In [15]:
df.head()

,tweet_id,user_id,full_text,hashtags
0,1249125075179016192,1047598652510937088,App de COVID-19 pudo ser un mero sitio web inf...,[]
1,1249125077020532740,233033221,Desafíos y oportunidades por el COVID-19 https...,[]
2,1249125082171146246,3433643799,Ganamos el ultimo UHC del evento Benefico cont...,[]
3,1249125108284882945,1413747024,"En toda circunstancia, en todo lugar, siempre ...",[]
4,1249125110398795782,15666807,El epidemiólogo Abdel Rodriguez Rivera habla c...,[]


In [16]:
from collections import Counter

In [17]:
ht_counts = Counter()

In [18]:
for hts in df.hashtags.values:
    ht_counts.update(hts)

In [19]:
ht_counts.most_common(100)

[("b'covid19'", 56322),
 ("b'quedateencasa'", 8892),
 ("b'coronavirus'", 6868),
 ("b'covid'", 3284),
 ("b'cuba'", 2710),
 ("b'covid_19'", 1815),
 ("b'atencionmedicadecalidad'", 1303),
 ("b'cubasalvavidas'", 1256),
 ("b'venezuela'", 1227),
 ("b'estevirusloparamosunidos'", 1185),
 ("b'yomequedoencasa'", 1041),
 ("b'eeuu'", 931),
 ("b'laprevencioneslaclave'", 895),
 ("b'yoapoyolacuarentena'", 856),
 ("b'ultimahora'", 809),
 ("b'covid19mx'", 803),
 ("b'venezuelabellaenrevolucion'", 780),
 ("b'17abr'", 761),
 ("b'cuarentena'", 739),
 ("b'concienciaycompromiso'", 685),
 ("b'mexico'", 641),
 ("b'17anosbarrioadentro'", 603),
 ("b'abrildeunioncivicomilitar'", 598),
 ("b'cubaporlasalud'", 564),
 ("b'abrildevictoriapopular'", 545),
 ("b'atencion'", 538),
 ("b'pandemia'", 529),
 ("b'covid2019'", 512),
 ("b'ahora'", 506),
 ("b'urgente'", 506),
 ("b'15abr'", 506),
 ("b'envivo'", 500),
 ("b'reporte'", 500),
 ("b'covid19chile'", 498),
 ("b'envideo'", 489),
 ("b'sanadistancia'", 485),
 ("b'18abr'", 480

In [20]:
len(df)

184593

# BERTopic

In [21]:
from sentence_transformers import SentenceTransformer

In [22]:
# Spanish multilingual model (lightweight)
# This takes ages!
# model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [23]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [24]:
embeddings = model.encode(docs, batch_size=128, show_progress_bar=True)

Batches:   0%|          | 0/1443 [00:00<?, ?it/s]

In [87]:
import numpy as np
np.save("data/covid_twitter_sample_topic_model_embeddings.npy", embeddings)

In [25]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired

# Create your representation model
representation_model = KeyBERTInspired()

In [26]:
docs_sample = docs

In [27]:
embeddings_sample = embeddings

In [28]:
embeddings_sample.shape

(184587, 384)

In [29]:
# Create and fit the BERTopic model using the precomputed embeddings
topic_model = BERTopic(language="spanish")  # or language="spanish", but multilingual works well

# topics, _ = topic_model.fit_transform(docs, embeddings)
topics, _ = topic_model.fit_transform(docs_sample, embeddings_sample)

# View topic info
topic_info = topic_model.get_topic_info()


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitl

In [30]:
topic_info.head(20)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,62821,-1_te_nos_lo_tiempos,"[te, nos, lo, tiempos, medidas, quedateencasa,...",[Pregunta seria para quien sepa dar una respue...
1,0,1127,0_concientizador_peda_saldría_grabar,"[concientizador, peda, saldría, grabar, vacaci...",[Ayer fue mi primer guardia en un área COVID e...
2,1,942,1_pandemia_7kkkxzmxgo_pandemias_desgarradores,"[pandemia, 7kkkxzmxgo, pandemias, desgarradore...",[Llamó a no generar estado de alarma difundien...
3,2,541,2_educación_escolar_clases_estudiantes,"[educación, escolar, clases, estudiantes, curs...",[La ministra de @educaciongob ha señalado que ...
4,3,415,3_296_449_546_406,"[296, 449, 546, 406, 847, 399, 297, 273, 875, ...",[Actualización sobre el #covid19 en México; Sa...
5,4,408,4_dezurdateam_diazcanelb_cubaporlasalud_cubasa...,"[dezurdateam, diazcanelb, cubaporlasalud, cuba...",[@DeZurdaTeam @DiazCanelB @marti160patria @Sec...
6,5,397,5_xrbrsdzzsf_calificada_estatus_rigor,"[xrbrsdzzsf, calificada, estatus, rigor, angel...","[🇩🇪 Jefa de Gobierno y, por si fuera poco, cie..."
7,6,391,6_y6tdcl9ksp_wf73icot24_soydeaquí_tengp7rxny,"[y6tdcl9ksp, wf73icot24, soydeaquí, tengp7rxny...",[Favor contestar y difundir! 👇🏽👇🏽👇🏽 https://t....
8,7,380,7_mujeres_violencia_género_feminista,"[mujeres, violencia, género, feminista, machis...","[Hoy, en el contexto de emergencia ante la #Co..."
9,8,365,8_cárceles_presos_hacinamiento_reclusos,"[cárceles, presos, hacinamiento, reclusos, exc...",[#LiberenALosPresosPolíticos | Exigimos la lib...


In [32]:
import spacy
import string

# Load the Spanish library from SpaCy
nlp = spacy.load("es_core_news_sm")

# Create list of punctuation marks
punctuations = string.punctuation

# Create list of stopwords from spaCy
es_stopwords = spacy.lang.es.stop_words.STOP_WORDS

stopwords = list(es_stopwords) + ["https", "http", "com", "covid", "covid19", "19", "co", "coronavirus", "rt"]

In [33]:
len(docs_sample)

184587

In [82]:
docs_sample

['App de COVID-19 pudo ser un mero sitio web informativo\n \nhttps://t.co/NkzRDGJNKa',
 'Desafíos y oportunidades por el COVID-19 https://t.co/nEajnYBi7L',
 'Ganamos el ultimo UHC del evento Benefico contra el COVID y la verdad que lo que mas importa es que recaudaron 3700 euros que es una banda asi que muchas gracias a todos.\nNo pude stremear por problemas de internet pero me hubiera encantado, gracias again. https://t.co/A9EwTnLVx5',
 'En toda circunstancia, en todo lugar, siempre la protección de Carabineros. Gracias una vez mas',
 'El epidemiólogo Abdel Rodriguez Rivera habla con NotiCentro sobre recomendaciones ante la situación del Covid-19. https://t.co/MrioyQSRKg',
 'El señor que murió por Covid-19 en Caracas no viajó en 1 año, no se contagió en el exterior, se enfermó aquí en Vzla y NO sabemos de quien, por favor dejen de desestimar la situacion mucha gente puede tener el virus y ser completamente asintomático.',
 'Bro, qué coño le hicieron a La Palma, Tenerife, El Hierro y L

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

# Fine-tune topic representations after training BERTopic
vectorizer_model = CountVectorizer(stop_words=stopwords, ngram_range=(1, 3), min_df=10)
topic_model.update_topics(docs_sample, vectorizer_model=vectorizer_model)
info = topic_model.get_topic_info()

In [79]:
# Method 1 - safetensors
embedding_model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
topic_model.save("covid_twitter_topic_model", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)


In [80]:
topic_model_loaded = BERTopic.load("covid_twitter_topic_model")

In [81]:
topic_model_loaded.get_topic_info().head(20)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,62821,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",NaN
1,0,1127,0_grabar_vacaciones_siento_mar,"[grabar, vacaciones, siento, mar, podido, hubi...",NaN
2,1,942,1_pandemia_pandemia covidー19_frente pandemia_p...,"[pandemia, pandemia covidー19, frente pandemia,...",NaN
3,2,541,2_educación_escolar_estudiantes_curso,"[educación, escolar, estudiantes, curso, clase...",NaN
4,3,415,3_casos méxico_296_449_546,"[casos méxico, 296, 449, 546, 847, 406, 297, 2...",NaN
5,4,408,4_dezurdateam_diazcanelb_cubaporlasalud_diazca...,"[dezurdateam, diazcanelb, cubaporlasalud, diaz...",NaN
6,5,397,5_estatus_rigor_angela_canciller,"[estatus, rigor, angela, canciller, jefa gobie...",NaN
7,6,391,6_contestar_leer_mantente_ke,"[contestar, leer, mantente, ke, echa, informac...",NaN
8,7,380,7_género_violencia_mujeres_violencia género,"[género, violencia, mujeres, violencia género,...",NaN
9,8,365,8_cárceles_presos_hacinamiento_reclusos,"[cárceles, presos, hacinamiento, reclusos, lib...",NaN


In [61]:
info[info.Count > 100]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,62821,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...
1,0,1127,0_grabar_vacaciones_siento_mar,"[grabar, vacaciones, siento, mar, podido, hubi...",[Ayer fue mi primer guardia en un área COVID e...
2,1,942,1_pandemia_pandemia covidー19_frente pandemia_p...,"[pandemia, pandemia covidー19, frente pandemia,...",[Llamó a no generar estado de alarma difundien...
3,2,541,2_educación_escolar_estudiantes_curso,"[educación, escolar, estudiantes, curso, clase...",[La ministra de @educaciongob ha señalado que ...
4,3,415,3_casos méxico_296_449_546,"[casos méxico, 296, 449, 546, 847, 406, 297, 2...",[Actualización sobre el #covid19 en México; Sa...
...,...,...,...,...,...
111,110,103,110_bmv_ruiz sacristán_jaime ruiz_sacristán,"[bmv, ruiz sacristán, jaime ruiz, sacristán, r...","[Por Covid-19, muere el presidente de la BMV, ..."
112,111,102,111_hacerle frente_precedentes_traer_hacerle,"[hacerle frente, precedentes, traer, hacerle, ...","[En una operación sin precedentes, hoy volamos..."
113,112,101,112_importados_medidas contención_activista_ca...,"[importados, medidas contención, activista, ca...",[📌 El artista y activista chino Ai Weiwei afir...
114,113,101,113_alquiler_vivienda_viviendas_renta,"[alquiler, vivienda, viviendas, renta, económi...",[Adivinen qué país tiene una suspensión de alq...


,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,App de COVID-19 pudo ser un mero sitio web inf...,-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False
1,Desafíos y oportunidades por el COVID-19 https...,-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False
2,Ganamos el ultimo UHC del evento Benefico cont...,2711,2711_1000_chicos_surgió_euros,"[1000, chicos, surgió, euros, concurso, ganamo...",[Muchas gracias chicos de esta mamarrachada de...,1000 - chicos - surgió - euros - concurso - ga...,1.000000,False
3,"En toda circunstancia, en todo lugar, siempre ...",-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False
4,El epidemiólogo Abdel Rodriguez Rivera habla c...,683,683_lorenzo mendoza_mendoza_escoltas_lorenzo,"[lorenzo mendoza, mendoza, escoltas, lorenzo, ...",[¡ERA DE ESPERARSE! Lorenzo Mendoza podría ten...,lorenzo mendoza - mendoza - escoltas - lorenzo...,0.400433,False
...,...,...,...,...,...,...,...,...
184582,#SociedadPA Anoche salió del Aeropuerto Intern...,-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False
184583,Presidente Maduro reporta siete nuevos casos d...,319,319_nicolás maduro_nicolás_maduro_reporta casos,"[nicolás maduro, nicolás, maduro, reporta caso...",[Nicolás Maduro reporta 4 nuevos casos positiv...,nicolás maduro - nicolás - maduro - reporta ca...,0.317041,False
184584,¿Por qué al hacer compra millonaria de pruebas...,1328,1328_expresamos_generan_correspondientes_solicitó,"[expresamos, generan, correspondientes, solici...",[¿Por qué al hacer compra millonaria de prueba...,expresamos - generan - correspondientes - soli...,0.713132,True
184585,Todos coludos o todos rabones.#GobiernoDeMentiras,-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False


In [43]:
res = topic_model.get_document_info(docs_sample).query("Topic != -1 & Representative_document")[['Document', 'Topic']]

In [67]:
def save_examples(n_topic):
    topic_name = info.Name[info.Topic == n_topic].values[0]
    examples = get_examples(n_topic)
    print(topic_name)
    examples.to_csv(f"examples/{topic_name}.csv", index=False)

In [68]:
%%sh
mkdir -p examples

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [77]:
info.head(100)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,62821,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...
1,0,1127,0_grabar_vacaciones_siento_mar,"[grabar, vacaciones, siento, mar, podido, hubi...",[Ayer fue mi primer guardia en un área COVID e...
2,1,942,1_pandemia_pandemia covidー19_frente pandemia_p...,"[pandemia, pandemia covidー19, frente pandemia,...",[Llamó a no generar estado de alarma difundien...
3,2,541,2_educación_escolar_estudiantes_curso,"[educación, escolar, estudiantes, curso, clase...",[La ministra de @educaciongob ha señalado que ...
4,3,415,3_casos méxico_296_449_546,"[casos méxico, 296, 449, 546, 847, 406, 297, 2...",[Actualización sobre el #covid19 en México; Sa...
...,...,...,...,...,...
95,94,111,94_daniel ortega_ortega_daniel_nicaragua,"[daniel ortega, ortega, daniel, nicaragua, mur...","[#15Abril El imperio genocida del norte, la po..."
96,95,111,95_hlgatell lopezobrador__chantaje_acepta_temp...,"[hlgatell lopezobrador_, chantaje, acepta, tem...",[Intento de chantaje televisivo de @RicardoBSa...
97,96,110,96_comenzando_científicos chinos_280_viruschino,"[comenzando, científicos chinos, 280, viruschi...",[ATENCION | Científicos chinos están comenzand...
98,97,110,97_rusia_ruso_politizar_acusaciones,"[rusia, ruso, politizar, acusaciones, xi, most...",[El pensador ruso Mikhail Kovalchuk sostiene l...


In [78]:
info.to_csv("examples/topics.csv", index=False)

In [76]:
save_examples(36)

36_turismo_turistas_do_interno


In [75]:
save_examples(34)

34_mascarillas_facial_tela_mascarilla


In [74]:
save_examples(32)

32_decretada_fase pandemia_fase_mañana


In [72]:
save_examples(12)

12_vacuna_vacunas_desarrollo vacuna_probar


In [71]:
save_examples(2)

2_educación_escolar_estudiantes_curso


In [70]:
save_examples(1)

1_pandemia_pandemia covidー19_frente pandemia_pandemia país


In [69]:
save_examples(0)

0_grabar_vacaciones_siento_mar


In [ ]:
Probability

In [59]:
get_examples(12)

,Document,Representative_document,Probability
76046,"""Quería ser útil"": La primera persona en proba...",True,1.000000
79805,"""Quería ser útil"": La primera persona en proba...",True,1.000000
80179,"""Quería ser útil"": La primera persona en proba...",True,1.000000
81217,"""Quería ser útil"": La primera persona en proba...",True,1.000000
91474,"""Quería ser útil"": La primera persona en proba...",True,1.000000
...,...,...,...
52417,La Fundación de #BillGates y su esposa está fi...,False,0.586509
100764,"3) , aluminio y mercurio ... por tejido animal...",False,0.584316
93138,🌍 😷 💉\nLa OMS anuncia que se han iniciado los ...,False,0.583863
121115,El director de la Organización Mundial de la S...,False,0.583319


In [51]:
topic_model.get_document_info(docs_sample)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,App de COVID-19 pudo ser un mero sitio web inf...,-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False
1,Desafíos y oportunidades por el COVID-19 https...,-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False
2,Ganamos el ultimo UHC del evento Benefico cont...,2711,2711_1000_chicos_surgió_euros,"[1000, chicos, surgió, euros, concurso, ganamo...",[Muchas gracias chicos de esta mamarrachada de...,1000 - chicos - surgió - euros - concurso - ga...,1.000000,False
3,"En toda circunstancia, en todo lugar, siempre ...",-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False
4,El epidemiólogo Abdel Rodriguez Rivera habla c...,683,683_lorenzo mendoza_mendoza_escoltas_lorenzo,"[lorenzo mendoza, mendoza, escoltas, lorenzo, ...",[¡ERA DE ESPERARSE! Lorenzo Mendoza podría ten...,lorenzo mendoza - mendoza - escoltas - lorenzo...,0.400433,False
...,...,...,...,...,...,...,...,...
184582,#SociedadPA Anoche salió del Aeropuerto Intern...,-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False
184583,Presidente Maduro reporta siete nuevos casos d...,319,319_nicolás maduro_nicolás_maduro_reporta casos,"[nicolás maduro, nicolás, maduro, reporta caso...",[Nicolás Maduro reporta 4 nuevos casos positiv...,nicolás maduro - nicolás - maduro - reporta ca...,0.317041,False
184584,¿Por qué al hacer compra millonaria de pruebas...,1328,1328_expresamos_generan_correspondientes_solicitó,"[expresamos, generan, correspondientes, solici...",[¿Por qué al hacer compra millonaria de prueba...,expresamos - generan - correspondientes - soli...,0.713132,True
184585,Todos coludos o todos rabones.#GobiernoDeMentiras,-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False


In [54]:
def get_examples(n_topic):
    res = topic_model.get_document_info(docs_sample).query(f"Topic == {n_topic}")[['Document', 'Representative_document', 'Probability']]
    return res.sort_values(['Representative_document', 'Probability'], ascending=False)

In [56]:
df = get_examples(2)

94406     La ministra de @educaciongob ha señalado que n...
106668    La ministra de @educaciongob ha señalado que n...
112329    La ministra de @educaciongob ha señalado que n...
116172    La ministra de @educaciongob ha señalado que n...
107330             ¿Qué sería de nosotros sin la educación?
                                ...                        
123463    “Extraño la rutina de mi escuela, pero sé que ...
147647    Mi señora está haciendo una maestría en derech...
91312     🗣 "Ningún alumno perderá el curso escolar por ...
21392     #NuevaYork anuncia el cierre de colegios públi...
156746    #CadaFamiliaUnaEscuela\n#16Abr | Estudiante de...
Name: Document, Length: 541, dtype: object

In [ ]:
info.Representative_Docs[3]

In [ ]:
info.shape